In [1]:
import pandas as pd
import numpy as np
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD

In [2]:
import zipfile

In [3]:
path = r'C:\Users\rohan\test\test.csv' 
train = pd.read_csv(path)

In [4]:
train.head()

,id
0,60001
1,60002
2,60003
3,60004
4,60005


In [ ]:
train_img = []
path=r'C:\Users\rohan\test\test'
for img_name in tqdm(train['id']):
    # defining the image path
    image_path =  path+ '\\' + str(img_name) + '.png'
    # reading the image
    img = imread(image_path, as_gray=True)
    # normalizing the pixel values
    img /= 255.0
    # converting the type of pixel to float 32
    img = img.astype('float32')
    # appending the image into the list
    train_img.append(img)

In [ ]:
print(len(train_img))

In [ ]:
train_x=np.array(train_img)
train_y=train['label'].values
train_x.shape

In [ ]:
i=0
plt.figure(figsize=(10,10))
plt.subplot(221), plt.imshow(train_x[i], cmap='gray')
plt.subplot(222), plt.imshow(train_x[i+25], cmap='gray')
plt.subplot(223), plt.imshow(train_x[i+50], cmap='gray')
plt.subplot(224), plt.imshow(train_x[i+100], cmap='gray')

In [ ]:
train_x, val_x, train_y, val_y=train_test_split(train_x,train_y, test_size=0.1)
(train_x.shape, train_y.shape), (val_x.shape, val_y.shape)

In [ ]:
print(type(train_x))

In [ ]:
train_x=train_x.reshape(54000,1,28,28)
train_x=torch.from_numpy(train_x)
train_y=train_y.astype(int)
train_y=torch.from_numpy(train_y)


In [ ]:
val_x=val_x.reshape(6000,1,28,28)
val_x=torch.from_numpy(val_x)
val_y=val_y.astype(int)
val_y=torch.from_numpy(val_y)

In [ ]:
class Net(Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            Conv2d(1, 4, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(4),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            # Defining another 2D convolution layer
            Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(4),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
        )

        self.linear_layers = Sequential(
            Linear(4 * 7 * 7, 10)
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [ ]:
model = Net()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.07)
# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    
print(model)

In [ ]:
def train(epoch):
    model.train()
    tr_loss = 0
    # getting the training set
    x_train, y_train = Variable(train_x), Variable(train_y)
    # getting the validation set
    x_val, y_val = Variable(val_x), Variable(val_y)
    # converting the data into GPU format
    if torch.cuda.is_available():
        x_train = x_train.cuda()
        y_train = y_train.cuda()
        x_val = x_val.cuda()
        y_val = y_val.cuda()

    # clearing the Gradients of the model parameters
    optimizer.zero_grad()
    
    # prediction for training and validation set
    output_train = model(x_train)
    output_val = model(x_val)

    # computing the training and validation loss
    loss_train = criterion(output_train, y_train)
    loss_val = criterion(output_val, y_val)
    train_losses.append(loss_train)
    val_losses.append(loss_val)

    # computing the updated weights of all the model parameters
    loss_train.backward()
    optimizer.step()
    tr_loss = loss_train.item()
    if epoch%2 == 0:
        # printing the validation loss
        print('Epoch : ',epoch+1, '\t', 'loss :', loss_val)


In [ ]:
print(type(y_train))

In [ ]:
n_epochs=25
train_losses=[]
val_losses=[]
for epoch in range(n_epochs):
    train(epoch)

In [ ]:
plt.plot(train_losses, label='Training Loses')
plt.plot(val_losses, label='Validation Losses')
plt.legend()
plt.show()


In [ ]:
with torch.no_grad():
    output=model(train_x.cuda())

softmax=torch.exp(output).cpu()
prob=list(softmax.numpy())
predictions =np.argmax(prob, axis=1)
accuracy_score(train_y,predictions)

In [ ]:
with torch.no_grad():
    output=model(val_x.cuda())

softmax=torch.exp(output).cpu()
prob=list(softmax.numpy())
predictions =np.argmax(prob, axis=1)
accuracy_score(val_y,predictions)